## make training/testing dataset from all the articles

In [1]:
import os
import numpy as np
import pandas as pd
from itertools import combinations, permutations

def remove_suffix(input_string, suffix):
    if suffix and input_string.endswith(suffix):
        return input_string[:-len(suffix)]
    return input_string


def make_table(path):
    # read file names(id)
    list_of_files = os.listdir(path) # len=560
    index_list = []
    for file_name in list_of_files:
        # index_list.append(int(file_name.removesuffix('.txt')))  # for python 3.9+
        index_list.append(int(remove_suffix(file_name, '.txt')))

    # read file content
    titles = []
    texts = []
    notices = []
    for file in list_of_files:
        f = open("{}/{}".format(path, file), "r")
        temp = f.readlines()
        titles.append(temp[0])
        texts.append(temp[1])
        try:
            notices.append(temp[2])
        except:
            notices.append('')
        f.close()

    # make general table
    df_dirty = pd.DataFrame(
        {'ID': index_list,
         'title': titles,
         'text': texts,
         'notice': notices
        })
    return df_dirty



def make_data(dataType, table):
    
    # make text_pairs, data scope can be changed, I only include "title" here
    # number of text_pairs = Pn取2, train:n=559; test:n=421

    combs = permutations(table[['ID', 'title']].values, r=2)
    combs = pd.DataFrame([[s for s in comb] for comb in combs], columns=['text_a', 'text_b'])
    temp_l = pd.DataFrame(combs['text_a'].to_list(), columns = ['id_text_a', 'text_a'])
    temp_r = pd.DataFrame(combs['text_b'].to_list(), columns = ['id_text_b', 'text_b'])
    combs = pd.concat([temp_l, temp_r], axis=1)
    

    if dataType=='train':
        # make label set
        labels = pd.read_csv('TrainLabel.csv')
        pairs = set()
        for _, row in labels.iterrows():
            temp = frozenset([row['Test'], row['Reference']])
            if temp not in pairs:
                pairs.add(temp)

        # mark label on text_pairs
        combs['label'] = 'unlike'
        for ind, row in combs.iterrows():
            if {row['id_text_a'], row['id_text_b']} in pairs:
                combs.at[ind, 'label'] = 'like'

    
    # combs.to_csv("{}.tsv".format(dataType), sep="\t", index=False)            
    return combs


In [2]:
table_train = make_table("./dataTrainComplete")
data_train = make_data('train', table_train)
data_train = data_train[['text_a', 'text_b', 'label']]
data_train.to_csv("train.tsv", sep="\t", index=False)


table_test = make_table("./dataPublicComplete")
data_test = make_data('test', table_test)
data_test = data_test[['text_a', 'text_b', 'id_text_a', 'id_text_b']]
data_test.to_csv("test.tsv", sep="\t", index=False)



In [6]:
table_train.head(10)

,ID,title,text,notice
0,1301,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,一、近期時有細雨，加上早晚溫度低、露水量多，有利香蕉葉部病菌之感染及漫延，造成葉片嚴重發病及...,噴藥方式及藥劑配方如下列：(一)、肩背式微粒動力噴霧機：每公頃80%鋅錳乃浦可濕性粉劑2.3...
1,924,二期作水稻已陸續完成插秧，請留意防治水稻水象鼻蟲\n,二期作水稻已陸續插秧，請留意防治水稻水象鼻蟲行政院農業委員會桃園區農業改良場表示，目前二期作...,\n
2,595,鹿野鄉局部地區水稻胡麻葉枯病發生嚴重，臺東場籲請該地區農民及時防治\n,臺東區農業改良場（以下簡稱臺東場）表示，該場於進行水稻病蟲害監測調查及相關試驗時，發現鹿野鄉...,防治方法及應注意事項：一、施行稻種消毒；勿選用罹病老熟秧苗，以免傳播病原。二、發病嚴重田，於...
3,438,臺南區農改場籲請農友注意水稻白葉枯病、稻熱病、紋枯病與葉鞘腐敗病之發生與防治。臺南區農改場、...,水稻已至白葉枯病發病高峰期，田間也陸續發生白葉枯病，且稻熱病與紋枯病持續發展，臺南區農改場籲...,農友應注意葉稻熱病、紋枯病與白葉枯病在田間是否發生，倘若發生應立即用藥防治。葉鞘腐敗病為水稻...
4,344,早晚溫差大濕度高，臺南場籲請農友加強水稻稻熱病與白葉枯病防治。\n,近日台南地區早晚溫差大濕度高，罹患水稻白葉枯病與葉稻熱病有增加之趨勢，二期水稻也進入孕穗至抽...,水稻白葉枯病的病原細菌可由水孔或傷口侵入，常在葉片邊緣形成波浪狀病斑，隨著葉脈往下蔓延，在葉...
5,746,加強杜絕洋香瓜病毒病蔓延\n,雲嘉南地區洋香瓜目前正值生產季節，由於降雨量少，小型昆蟲包括蚜蟲、薊馬、銀葉粉蝨密度持續增加...,\n
6,1382,稻熱病蠢蠢欲動，高雄區農業改良場籲請農友及早防範\n,行政院農業委員會高雄區農業改良場（以下簡稱高雄場）表示，目前高屏地區日夜溫差較大，已逐漸出現...,稻熱病的防治方法，應依據植物保護手冊規定，選用適當藥劑進行施用防治，例如15%加普胺水懸劑、...
7,453,目前正值稻熱病流行高峰期，花蓮區農業改良場籲請農友注意加強防治\n,一、發生地區及防治適期：花蓮縣水稻栽培區大部份已有葉稻熱病發生，應注意田間發病情形並及時施藥...,\n
8,984,台中區農業改良場發佈水稻白葉枯病發生警報，呼籲轄區內農友發動共同防治工作，以免農作物遭受病蟲...,糧食作物病蟲害發生警報中華民國90年9月24日第肆號臺中區農業改良場彰化縣大村鄉松槐路370...,\n
9,1271,香蕉所籲請蕉農務必做好開花蕉株果房疏果、整梳及套袋工作，以提高蕉果品質。\n,一、蕉株生育期間適時防治病蟲害，可減少蕉葉感染病蟲害機會，以維持蕉果正常生長。二、疏果整梳及...,三、疏果、整梳方法：(一)、香蕉終花前分二~三次摘除蕉花，以減少蕉乳污染，並同時進行疏果、整...


In [3]:
data_train.head(10)

,text_a,text_b,label
0,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,二期作水稻已陸續完成插秧，請留意防治水稻水象鼻蟲\n,unlike
1,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,鹿野鄉局部地區水稻胡麻葉枯病發生嚴重，臺東場籲請該地區農民及時防治\n,unlike
2,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,臺南區農改場籲請農友注意水稻白葉枯病、稻熱病、紋枯病與葉鞘腐敗病之發生與防治。臺南區農改場、...,unlike
3,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,早晚溫差大濕度高，臺南場籲請農友加強水稻稻熱病與白葉枯病防治。\n,unlike
4,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,加強杜絕洋香瓜病毒病蔓延\n,unlike
5,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,稻熱病蠢蠢欲動，高雄區農業改良場籲請農友及早防範\n,unlike
6,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,目前正值稻熱病流行高峰期，花蓮區農業改良場籲請農友注意加強防治\n,unlike
7,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,台中區農業改良場發佈水稻白葉枯病發生警報，呼籲轄區內農友發動共同防治工作，以免農作物遭受病蟲...,unlike
8,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,香蕉所籲請蕉農務必做好開花蕉株果房疏果、整梳及套袋工作，以提高蕉果品質。\n,unlike
9,香蕉所表示近日低溫細雨香蕉葉部病害易發生，防檢局籲請農友加強注意病徵，以及時進行防治\n,水稻水象鼻蟲及螟蟲(二化螟、三化螟)已於田間發生，請加強防治措施，避免損失。\n,unlike


In [4]:
data_test.head(10)

,text_a,text_b,id_text_a,id_text_b
0,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,香蕉所籲請蕉農加強香蕉黑星病及葉斑病防治作業，以確保香蕉良好之品質及產量\n,128,1285
1,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,水稻秧苗病害管理\n,128,927
2,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,春季陰晴不定，請農友儘早預防稻熱病。臺東農改場、防檢局及田邊好幫手關心您。\n,128,294
3,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,時值外銷季節及夏蕉抽穗期，籲請蕉農做好開花蕉株果房疏果、整梳、套袋及蕉園灌溉、排水工作，俾確...,128,1351
4,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,請宣導蕉農做好香蕉黑星病防治作業，以確保香蕉良好之品質及產量。\n,128,1216
5,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,銀葉粉蝨-小害蟲，大災害 花蓮場籲請農民加強防範\n,128,462
6,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,近日氣溫逐漸升高，臺東場籲請釋迦農友加強小黃薊馬防治\n,128,270
7,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,芒果花期小型昆蟲密度持續上升，臺南場籲加強防治。\n,128,350
8,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,發布新聞稿提醒農民加強防治水稻褐飛蝨及縱捲葉蟲\n,128,617
9,水稻瘤野螟及褐飛蝨開始出現，請加強注意並及時防範，以避免損失。\n,連日降雨，高雄區農改場籲請農友小心防範甘藍黑腐病及黑斑病\n,128,1388


## if you want to make smaller dataset and control the like/unlike ratio

In [7]:
from sklearn.utils import shuffle

# iloc 自己調
temp_like = data_train.loc[data_train['label']=='like'][['text_a', 'text_b', 'label']].iloc[50:]
temp_unlike = data_train.loc[data_train['label']=='unlike'][['text_a', 'text_b', 'label']].iloc[50:]

dataType = 'train'  # or 'test'
shuffle(pd.concat([temp_like, temp_unlike], axis=0)).to_csv("{}_small.tsv".format(dataType), sep='\t', index=False)
